In [141]:
from collections import Counter
import numpy as np

In [142]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4


In [143]:
def valid_moves(board):
#Returns columns where a disc may be played
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]

def play(board, column, player):
#Updates `board` as `player` drops a disc in `column`\"\"\"\n",
   (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
   board[column, index] = player

def take_back(board, column):
#Updates `board` removing top disc from `column`\"\"\"\n",
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0
    
def four_in_a_row(board, player):
    #Checks if `player` has a 4-piece line\"\"\"\n",
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
            )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                    for ro in range(0, NUM_COLUMNS - FOUR + 1)
                    for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
                np.all(board[diag] == player)
                for diag in (
                    (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                    for ro in range(0, NUM_COLUMNS - FOUR + 1)
                    for co in range(0, COLUMN_HEIGHT - FOUR + 1)
                )
        )
    )

In [144]:
def _mcSquillero(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarloSquillero(board, player):
    montecarlo_samples = 10
    cnt = Counter(_mcSquillero(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples

def tryMove(board, player):
    return montecarloSquillero(board, player)


def montecarloEval (board, c, player):
    p=-player
    #even number of moves required to the analisys of optimal response
    boardTmp = np.copy(board)
    play(boardTmp, c, player)
    f = 0

    for f in valid_moves(board):
        p=-p
        res = tryMove(boardTmp, p)

    return res


def _mc(board, player):
    p = -player
    x = 0
    best = -player
    col = 0
    
    for c in valid_moves(board):
        p=-p
        tmp = montecarloEval(board, c, p)*player
        if best < tmp :
            best = tmp
            col = c
        
    play(board, col, p)
    if four_in_a_row(board, p):
        return p
    return 0    
    
def eval_board(board, player):
    if four_in_a_row(board, 1):
    # Alice won
        return 1
    elif four_in_a_row(board, -1):
    # Bob won
        return -1
    else:
    # Not terminal, let's simulate...
        return _mc(board, player)


In [145]:
slots = NUM_COLUMNS*COLUMN_HEIGHT
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1,)
play(board, 2, -1)
play(board, 4, 1)
play(board, 0, -1)

print(board)
eval_board(board, 1)
print(board)

[[-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
[[-1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]]
